In [1]:
import os
import shlex, subprocess
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from lsst.resources import ResourcePath

In [1]:
def getMainHeaderInfo(hdu_list):
    hdr0 = hdu_list[0].header
    roiCols = hdr0['ROICOLS']
    roiRows = hdr0['ROIROWS']
    try:
        roiUnder = hdr0['ROIUNDRC']
    except:
        roiUnder = 6
    nStamps = hdr0['N_STAMPS']
    
    # Set the xor value - Guider CCDs are different from science CCDs
    if raft in ['R00', 'R04', 'R40', 'R44']:
        # Guider rafts
        xor = 0x20000
    else:
        # Science rafts
        xor = 0x1ffff
    return [roiRows, roiCols, roiUnder, nStamps, xor]

def unpackStamps(hduNum):
    data = np.array(hdu_list[hduNum].data[0]).astype('>u4')[0]
    data.byteswap(inplace=True)
    totalCols = roiCols + roiUnder
    size = roiRows * totalCols
    out = np.zeros([16, size], dtype=int)
    image_out = np.zeros([16, roiRows, roiCols], dtype=int)
    for n in range(size):
        # Get 9 32 bit words of data
        res = ''
        for i in range(9):
            d = data[(size - n) * 9 - i - 1]
            d = format(d, '#034b')
            d = d.split('b')[1]
            res += d
        # Now extract 16 18 bit words from the data
        for i in range(16):
            bin_value = res[i * 18:(i + 1) * 18]
            int_value = int(bin_value, 2)
            final_value = int_value ^ xor
            out[i,n] = final_value  
    for i in range(16):
        reshaped = out[i,:].reshape(roiRows, totalCols)
        image_out[i,:,:] = np.flipud(np.fliplr(reshaped[:,0:roiCols]))
    return image_out

In [2]:
# Set the scaling
autoscale = False
# Scale to use if autoscale = False
vmin = 14500
vmax = 15000
# Set dayObs, seqNum
dayObs = 20250410
seqNum = 8

In [6]:
expId = int(f"{dayObs}{seqNum:05d}")

# Get one CCD to know nStamps
raft = 'R00'
ccd = 'SG0'
filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"

rp = ResourcePath(filename)
with rp.open(mode="rb") as f:
    hdu_list = fits.open(f)
[roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)

# This defines the plot locations for the 8 CCDs
# Each entry is [raft, ccd, i0, j0, di, dj, rot]
config = [['R00', 'SG0', 20, 9, -1, -1, True], ['R00', 'SG1', 12, 0, -1, 1, False], 
          ['R04', 'SG0', 11, 20, 1, -1, False], ['R04', 'SG1', 20, 12, -1, -1, True], 
          ['R40', 'SG0', 9, 0, -1, 1, False], ['R40', 'SG1', 0, 8, 1, 1, True],
          ['R44', 'SG0', 0, 11, 1, 1, True], ['R44', 'SG1', 8, 20, 1, -1, False]]

# dirName = f"/home/c/cslage/u/Guider_Mode/LSSTCam_movie_{expId}"
# %mkdir -p {dirName}
movieName = f"Guider_{expId}.mp4"
print(movieName)
# Build the individual frames
imArray = []
fig = plt.figure(figsize=(10,10))
for n in range(1, nStamps+1):
    axs = fig.subplots(21,21)
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    # Clear the axes and plot frames
    for i in range(21):
        for j in range(21):
            axs[i][j].axis('off')
            axs[i][j].set_xticks([])
            axs[i][j].set_yticks([])
    for [raft, ccd, i0, j0, di, dj, rot] in config:
        filename = f"s3://embargo@rubin-summit/LSSTCam/{dayObs}/MC_O_{dayObs}_{seqNum:06d}/MC_O_{dayObs}_{seqNum:06d}_{raft}_{ccd}_guider.fits"
        
        rp = ResourcePath(filename)
        with rp.open(mode="rb") as f:
            hdu_list = fits.open(f)
        [roiRows, roiCols, roiUnder, nStamps, xor] = getMainHeaderInfo(hdu_list)
        hduNum = 2 * n + 1
        hdrn = hdu_list[hduNum].header
        timestamp = hdrn['STMPTIME']
        image_out = unpackStamps(hduNum)
        i = i0; j = j0
        # Set the CCD titles
        if rot:
            if dj < 0:
                axs[i0 + 3 * di][j0 - 1].set_title(f"{raft}_{ccd}", fontsize=12, rotation='vertical',x=-0.3,y=0.5)
            else:
                axs[i0 + 3 * di][j0].set_title(f"{raft}_{ccd}", fontsize=12, rotation='vertical',x=-0.4,y=-0.5)
        else:
            if di < 0:
                axs[i0 - 1][j0 + 3 * dj].set_title(f"{raft}_{ccd}", fontsize=12, loc='center')
            else:
                axs[i0][j0 + 3 * dj].set_title(f"{raft}_{ccd}", fontsize=12, loc='center')
        # Now plot the data
        for seg in range(16):
            print(raft, ccd, seg, i, j)
            arr = image_out[seg]
            if rot:
                arr = np.transpose(arr)
            if autoscale:
                im = axs[i][j].imshow(arr, interpolation='nearest', origin='upper', cmap='Greys')
            else:
                med = np.nanmedian(image_out[seg])
                vmin = med - 20
                vmax = med + 20
                im = axs[i][j].imshow(arr, interpolation='nearest', origin='upper', vmin=vmin, vmax=vmax, cmap='Greys')
            #axs[i][j].text(roiRows/2.0, roiCols/2.0,f"{seg}") # For segment check
            if rot:
                if seg == 7:
                    j += dj
                    i = i0
                else:
                    i += di
            else:
                if seg == 7:
                    i += di
                    j = j0
                else:
                    j += dj
    plt.suptitle(f"Guider mode {expId}, \n Frame {n+1} {timestamp}", fontsize=18) 
    imArray.append(fig) # Append something here
    # plt.savefig(f"{dirName}/Frame_{n:03d}.png")
    plt.clf()
    if n % 10 == 0:
        print(f"Finished frame {n}")
print("Done building frames")

Guider_2025041000008.mp4


R00 SG0 0 20 9
R00 SG0 1 19 9
R00 SG0 2 18 9
R00 SG0 3 17 9
R00 SG0 4 16 9
R00 SG0 5 15 9
R00 SG0 6 14 9
R00 SG0 7 13 9
R00 SG0 8 20 8
R00 SG0 9 19 8
R00 SG0 10 18 8
R00 SG0 11 17 8
R00 SG0 12 16 8
R00 SG0 13 15 8
R00 SG0 14 14 8
R00 SG0 15 13 8


R00 SG1 0 12 0
R00 SG1 1 12 1
R00 SG1 2 12 2
R00 SG1 3 12 3
R00 SG1 4 12 4
R00 SG1 5 12 5
R00 SG1 6 12 6
R00 SG1 7 12 7
R00 SG1 8 11 0
R00 SG1 9 11 1
R00 SG1 10 11 2
R00 SG1 11 11 3
R00 SG1 12 11 4
R00 SG1 13 11 5
R00 SG1 14 11 6
R00 SG1 15 11 7


R04 SG0 0 11 20
R04 SG0 1 11 19
R04 SG0 2 11 18
R04 SG0 3 11 17
R04 SG0 4 11 16
R04 SG0 5 11 15
R04 SG0 6 11 14
R04 SG0 7 11 13
R04 SG0 8 12 20
R04 SG0 9 12 19
R04 SG0 10 12 18
R04 SG0 11 12 17
R04 SG0 12 12 16
R04 SG0 13 12 15
R04 SG0 14 12 14
R04 SG0 15 12 13


R04 SG1 0 20 12
R04 SG1 1 19 12
R04 SG1 2 18 12
R04 SG1 3 17 12
R04 SG1 4 16 12
R04 SG1 5 15 12
R04 SG1 6 14 12
R04 SG1 7 13 12
R04 SG1 8 20 11
R04 SG1 9 19 11
R04 SG1 10 18 11
R04 SG1 11 17 11
R04 SG1 12 16 11
R04 SG1 13 15 11
R04 SG1 14 14 11
R04 SG1 15 13 11


R40 SG0 0 9 0
R40 SG0 1 9 1
R40 SG0 2 9 2
R40 SG0 3 9 3
R40 SG0 4 9 4
R40 SG0 5 9 5
R40 SG0 6 9 6
R40 SG0 7 9 7
R40 SG0 8 8 0
R40 SG0 9 8 1
R40 SG0 10 8 2
R40 SG0 11 8 3
R40 SG0 12 8 4
R40 SG0 13 8 5
R40 SG0 14 8 6
R40 SG0 15 8 7


R40 SG1 0 0 8
R40 SG1 1 1 8
R40 SG1 2 2 8
R40 SG1 3 3 8
R40 SG1 4 4 8
R40 SG1 5 5 8
R40 SG1 6 6 8
R40 SG1 7 7 8
R40 SG1 8 0 9
R40 SG1 9 1 9
R40 SG1 10 2 9
R40 SG1 11 3 9
R40 SG1 12 4 9
R40 SG1 13 5 9
R40 SG1 14 6 9
R40 SG1 15 7 9


R44 SG0 0 0 11
R44 SG0 1 1 11
R44 SG0 2 2 11
R44 SG0 3 3 11
R44 SG0 4 4 11
R44 SG0 5 5 11
R44 SG0 6 6 11
R44 SG0 7 7 11
R44 SG0 8 0 12
R44 SG0 9 1 12
R44 SG0 10 2 12
R44 SG0 11 3 12
R44 SG0 12 4 12
R44 SG0 13 5 12
R44 SG0 14 6 12
R44 SG0 15 7 12


R44 SG1 0 8 20
R44 SG1 1 8 19
R44 SG1 2 8 18
R44 SG1 3 8 17
R44 SG1 4 8 16
R44 SG1 5 8 15
R44 SG1 6 8 14
R44 SG1 7 8 13
R44 SG1 8 9 20
R44 SG1 9 9 19
R44 SG1 10 9 18
R44 SG1 11 9 17
R44 SG1 12 9 16
R44 SG1 13 9 15
R44 SG1 14 9 14
R44 SG1 15 9 13
Done building frames


<Figure size 1000x1000 with 0 Axes>

In [ ]:
ani = animation.ArtistAnimation(fig=fig, artists=imArray, interval=400)
plt.show()

In [ ]:
# print(f"\033[1mThe movie name will be: {dirName}/{movieName}\033[0m")

# command = f"ffmpeg -pattern_type glob -i '{dirName}/*.png' -f mp4 -vcodec libx264 -pix_fmt yuv420p -framerate 50 -y {dirName}/{movieName}"
# args = shlex.split(command)
# build_movie = subprocess.Popen(args)
# build_movie.wait()